In [1]:
from azureml.core import Workspace

In [2]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import LocalWebservice

In [3]:
ws = Workspace.from_config()

In [4]:
# Create inference configuration based on the environment definition and the entry script
myenv = Environment.get(ws, "diabetes-env")

In [8]:
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

In [9]:
# ローカル環境で Debug
deployment_config = LocalWebservice.deploy_configuration(port=8890)

In [10]:
model = Model(ws, "diabetes-model")

In [11]:
local_service = Model.deploy(
    ws, "dia", [model], inference_config, deployment_config)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry f79f2067feb24fe5bd660f1f5123e456.azurecr.io
Logging into Docker registry f79f2067feb24fe5bd660f1f5123e456.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM f79f2067feb24fe5bd660f1f5123e456.azurecr.io/azureml/azureml_16a60a5e0afc11b3b7176c7e7784bb27
 ---> 097fa6aa5147
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 1f570cf6fbc7
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjgyYTVkOGQzLTUzMjItNGM0OS1iOWQ2LWRhNmUwMGJlNWQ1NyIsInJlc291cmNlR3JvdXBOYW1lIjoiYXp1cmVtbCIsImFjY291bnROYW1lIjoiYXp1cmVtbCIsIndvcmtzcGFjZUlkIjoiZjc5ZjIwNjctZmViMi00ZmU1LWJkNjYtMGYxZjUxMjNlNDU2In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in b82097672c05
 ---> 2817c0be9877
Step 4/5 : RUN mv '/var/azureml-app/tmpkf25vmsl.py' /var/azureml-app/main.py
 ---> Running in 02d1b2f56e07
 ---> 49e0

In [13]:
print(local_service.get_logs())

2020-12-29T13:46:11,694763454+00:00 - iot-server/run 
2020-12-29T13:46:11,694837155+00:00 - gunicorn/run 
2020-12-29T13:46:11,694823255+00:00 - rsyslog/run 
2020-12-29T13:46:11,696614073+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2020-12-29T13:46:12,023014116+00:00 - iot-server/finish 1 0
2020-12-29T13:46:12,024324329+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (10)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 41
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2020-12-29 13:46:14,964 | root | INFO | Starting up app insights client
Starting up app insights client
2020-12-29 13:46:14,965 | root | INFO | Starting up request id generator
Starting up request id generator
2020-12-29 13:46:14,965 | root | INFO | Starting up app insight hooks
Starting up app insight hooks
2020-12-29 13:46:14,965 | root | INFO | Invok

In [15]:
from azureml.core.webservice import AciWebservice

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name="test",
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

In [16]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............................
Succeeded
ACI service creation operation finished, operation "Succeeded"
